In [2]:
!pip install librosa
!pip install mirdata
!pip install smart_open
!pip install midiutil
!pip install PyGuitarPro==0.6
!pip install mido

!curl https://raw.githubusercontent.com/dada-bots/dadaGP/main/dadagp.py -o dadagp.py
!curl https://raw.githubusercontent.com/dada-bots/dadaGP/main/token_splitter.py -o token_splitter.py
!curl -LJO https://github.com/dada-bots/dadaGP/raw/main/blank.gp5

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 91943  100 91943    0     0   101k      0 --:--:-- --:--:-- --:--:--  101k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 23742  100 23742    0     0  49078      0 --:--:-- --:--:-- --:--:-- 49359
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

  0     0    0     0    0     0      0      0 --

In [6]:


folder = "../GuitarProConversor/tabs/"

import librosa
import numpy as np
import soundfile as sf
import math
import os
from midiutil import MIDIFile
import matplotlib.pyplot as plt
import json
import random

for file in os.listdir(folder):
  try:
      
    file_name = os.path.splitext(file)[0]
    token_file = 'tokens/' + file_name + '.tokens.txt'
    jams_file = 'jams/' + file_name + '.jams'
    midi_file = 'midis/' + file_name + '.mid'
    
    if os.path.exists(token_file) and os.path.exists(midi_file) and os.path.exists(jams_file) :
        print(f"Output files already exist for {file}. Skipping.")
        continue
      
    ################################################################### TOKENS - INSTRUCTIONS ####################################################################
    
    file_path = os.path.join(folder, file)
    token_file = 'tokens/'+ os.path.splitext(file)[0] + '.tokens.txt'  
    %run dadagp.py encode {file_path} {token_file} unknown


    listSegment = []
    oneSegment = {}


    with open(token_file) as file_in:

        for line in file_in:
            instructions = line.strip('\n').split(':')
            combinedInstructions = '\t'.join(instructions)
            # print(instructions)

            if 'tempo' in combinedInstructions and 'bfx' not in  combinedInstructions:
                # tempo = int(instructions[-1]) 
                tempo = random.randint(60,110)              #Random BPM
                oneSegment['tempo'] = tempo
                tempoMs = 60000 / tempo

            if 'distorted' in combinedInstructions or 'clean' in combinedInstructions:
                oneSegment['track'] = instructions[0]

            if 'nfx' in instructions:
                oneSegment.setdefault('techniques', []).append(instructions[-1])

            if oneSegment.get('track') in instructions and 'note' in instructions:
                if int(instructions[2].strip('s')) <= 6 and int(instructions[3].strip('f')) <= 19:
                    note = "".join([re.sub(r'\D', '', instructions[2]), '-', re.sub(r'\D', '', instructions[3])])
                    oneSegment.setdefault('notes', []).append(note)

            if  'rest' in instructions:
                oneSegment.setdefault('notes', []).append('rest')

            if 'wait' in instructions:
                duration = int(instructions[1])
                # print(duration)
                durationMs = math.trunc(duration/960.0 * 1000)/1000
                # print(durationMs)
                durationMs = durationMs * tempoMs
                # print(durationMs)
                oneSegment['duration'] = durationMs
                listSegment.append(oneSegment)
                oneSegment = {}

    notesForLibrosa = []
    for segment_dict in listSegment:
        notesForLibrosa.append({
            'tempo': segment_dict.get('tempo', ''),
            'track': segment_dict.get('track', ''),
            'notes': segment_dict.get('notes', []),
            'techniques': segment_dict.get('techniques', []),
            'duration': segment_dict.get('duration', '')
        })
        
    
    ############################################################## CREATE MIDI ###############################################################
    
    mf = MIDIFile(6)
    tracks = [0,1,2,3,4,5]
    GlobalTempo = notesForLibrosa[0]['tempo']
    time = 0    # start at the beginning
    mf.addTrackName(tracks[0], time, "MultiChannel MIDI")
    mf.addTempo(tracks[0], time, GlobalTempo)
    channel = 0
    volume = 100
    jam_data = []

    total_duration = 0
    guitarStrings = [64,59,55,50,45,40]

    array_test = []
    for event in notesForLibrosa:
        # print(event)

        if event['tempo'] != '':
            GlobalTempo = event['tempo']
            # print(GlobalTempo)
            # for index, item in enumerate(tracks):
            mf.addTempo(tracks[0], time, GlobalTempo)

        duration = event['duration']
        total_duration += event['duration']
        beat = duration / (60000 / GlobalTempo)
        beat = math.trunc(beat*1000)/1000
        # print(duration, beat)
        jam_data.append([event['notes'],duration, total_duration])
        duration = beat

        for note in event['notes']:

            if note != 'rest':
                string = int([note.split('-')][0][0])
                fret = int([note.split('-')][0][1])
                pitch = guitarStrings[string - 1] + fret
                mf.addNote(tracks[string-1], channel, pitch, time, duration, volume)
                array_test.append([fret,string,event['duration'],total_duration-event['duration']])

        time += event['duration'] / (60000 / GlobalTempo)

    with open('midis/' + os.path.splitext(file)[0] + '.mid', 'wb') as outf:
        mf.writeFile(outf)
    print('MIDI exported')
    print(len(array_test))
    
################################################################### CREATE JAM DATA #######################################################
    
    
    def generate_jams_from_data(data):
        jams_data = {
            "annotations": [],
            "file_metadata": {
                "title": "example",
                "artist": "",
                "release": "",
                "duration": None,
                "identifiers": {},
                "jams_version": "0.3.1"
            },
            "sandbox": {}
        }

        # Populate pitch_contour and note_midi data for each string
        for string_index in range(6):  # Assuming 6 guitar strings
            inverted_strings= [5, 4, 3, 2, 1, 0]
            pitch_contour_data = {
                "annotation_metadata": {
                    "curator": {"name": "", "email": ""},
                    "annotator": {},
                    "version": "",
                    "corpus": "",
                    "annotation_tools": "",
                    "annotation_rules": "",
                    "validation": "",
                    "data_source": str(inverted_strings[string_index])  # Change data_source value based on string index
                },
                "namespace": "pitch_contour",
                "data": {
                    "time": [],
                    "duration": [],
                    "value": [],
                    "confidence": []
                },
                "sandbox": {},
                "time": None,
                "duration": None
            }

            note_midi_data = {
                "annotation_metadata": {
                    "curator": {"name": "", "email": ""},
                    "annotator": {},
                    "version": "",
                    "corpus": "",
                    "annotation_tools": "",
                    "annotation_rules": "",
                    "validation": "",
                    "data_source": str(inverted_strings[string_index])  # Change data_source value based on string index
                },
                "namespace": "note_midi",
                "data": [],
                "sandbox": {},
                "time": None,
                "duration": None
            }
            

            for item in data:
                notes, duration, time = item
                time = time - duration
                time_sec = time / 1000  # Convert milliseconds to seconds

                # Populate pitch_contour data for the current string
                for note in notes:
                    if note != 'rest' and note != '':
                        string, fret = map(int, note.split('-'))
                        if string == string_index + 1:
                            string_note = guitarStrings[string - 1]
                            note_value = string_note + fret
                            pitch_contour_data["data"]["time"].append(time_sec)
                            pitch_contour_data["data"]["duration"].append(0.0)
                            pitch_contour_data["data"]["value"].append({"voiced": True, "index": 0, "frequency": librosa.midi_to_hz(note_value)})
                            pitch_contour_data["data"]["confidence"].append(None)

                # Populate note_midi data for the current string
                for note in notes:
                    if note != 'rest' and note != '':
                        string, fret = map(int, note.split('-'))
                        if string == string_index + 1:
                            string_note = guitarStrings[string - 1]
                            note_value = string_note + fret
                            note_midi_data["data"].append({"time": time_sec, "duration": duration / 1000, "value": note_value, "confidence": None})
                            
            # Set metadata and duration for pitch_contour and note_midi
            duration_seconds = max(item[2] for item in data) / 1000
            pitch_contour_data["time"] = duration_seconds
            pitch_contour_data["duration"] = duration_seconds
            note_midi_data["time"] = duration_seconds
            note_midi_data["duration"] = duration_seconds
    
            # Append pitch_contour and note_midi data for the current string to the annotations list
            jams_data["annotations"].append(pitch_contour_data)
            jams_data["annotations"].append(note_midi_data)

        # Set file duration
        tempo_data = {
                "annotation_metadata": {
                    "curator": {
                    "name": "",
                    "email": ""
                    },
                    "annotator": {},
                    "version": "",
                    "corpus": "",
                    "annotation_tools": "",
                    "annotation_rules": "",
                    "validation": "",
                    "data_source": ""
                },
                "namespace": "tempo",
                "data": [
                    {
                    "time": 0.0,
                    "duration": duration_seconds,
                    "value": notesForLibrosa[0]['tempo'],
                    "confidence": 1.0
                    }
                ],
                "sandbox": {},
                "time": duration_seconds,
                "duration": duration_seconds
                }
        jams_data["file_metadata"]["duration"] = duration_seconds
        jams_data['annotations'].append(tempo_data)

        return jams_data

    data = jam_data
    jams_data = generate_jams_from_data(data)

    # Save JAMS data to a file
    with open('jams/'+ os.path.splitext(file)[0] + '.jams', 'w') as f:
        json.dump(jams_data, f, indent=2)

    print("JAM file generated successfully.")

    
  except Exception as e:
    print(f"An error occurred while processing {file}: {e}")
    continue 




{'drums': [], 'distorted': [<guitarpro.models.Track object at 0x00000140AEB24C10>], 'clean': [], 'bass': [], 'leads': [], 'pads': [], 'remove': []}
Downtuning scheme of guitar/bass/pads/leads tracks: [0]
Pitch Shift: 0
{0: 'g6_standard'}
Head tokens
['unknown', 'downtune:0', 'tempo:120', 'start']
First 5 events:
[{'type': 'measure', 'track': -1, 'start': 960, 'tokens': ['new_measure']}, {'type': 'note', 'track': 0, 'instrument_prefix': 'distorted0', 'start': 960, 'duration': 240, 'string': 4, 'fret': 7, 'effects': ['nfx:let_ring']}, {'type': 'note', 'track': 0, 'instrument_prefix': 'distorted0', 'start': 960, 'duration': 240, 'string': 5, 'fret': 7, 'effects': ['nfx:let_ring']}, {'type': 'note', 'track': 0, 'instrument_prefix': 'distorted0', 'start': 960, 'duration': 240, 'string': 6, 'fret': 5, 'effects': ['nfx:let_ring']}, {'type': 'rest', 'track': 0, 'duration': 480, 'instrument_prefix': 'distorted0', 'start': 1200}]
First 20 tokens:
['unknown', 'downtune:0', 'tempo:120', 'start', '

In [7]:
import os
import json

directory = 'jams/'

# Function to rearrange annotations in each file
def rearrange_annotations(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        
    sorted_annotations = sorted(data['annotations'], key=lambda x: int(x['annotation_metadata']['data_source']) if x['annotation_metadata'].get('data_source') and x['annotation_metadata']['data_source'].isdigit() else float('inf'))

    with open(file_path, 'w') as file:
        json.dump({'annotations': sorted_annotations}, file, indent=2)

# Process each file in the directory to rearrange annotations
for filename in os.listdir(directory):
    if filename.endswith('.jams'):
        file_path = os.path.join(directory, filename)
        rearrange_annotations(file_path)

# Function to add metadata to each file
def add_metadata(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
        duration = data['annotations'][0]['duration']
        metadata = {
            "file_metadata": {
                "title": "example",
                "artist": "",
                "release": "",
                "duration": duration,
                "identifiers": {},
                "jams_version": "0.3.1"
            },
            "sandbox": {}
        }
        data.update(metadata)
        
        output_path = os.path.join(directory, os.path.splitext(os.path.basename(file_path))[0] + "_with_metadata.jams")
        with open(output_path, 'w') as output_file:
            json.dump(data, output_file, indent=4)

# Process each file in the directory to add metadata
for filename in os.listdir(directory):
    if filename.endswith('.jams'):
        file_path = os.path.join(directory, filename)
        add_metadata(file_path)

# Cleanup the directory, removing original files and renaming the new files
for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if 'with_metadata' not in filename:
        os.remove(file_path)
    else:
        new_filename = filename.replace('_with_metadata', '')
        new_file_path = os.path.join(directory, new_filename)
        os.rename(file_path, new_file_path)


In [86]:
import mirdata 

guitarSounds = mirdata.initialize('egfxset')
guitarSounds_ids = guitarSounds.track_ids

print(guitarSounds_ids)

['TapeEcho_Bridge-Middle/1-0', 'TapeEcho_Bridge-Middle/1-1', 'TapeEcho_Bridge-Middle/1-10', 'TapeEcho_Bridge-Middle/1-11', 'TapeEcho_Bridge-Middle/1-12', 'TapeEcho_Bridge-Middle/1-13', 'TapeEcho_Bridge-Middle/1-14', 'TapeEcho_Bridge-Middle/1-15', 'TapeEcho_Bridge-Middle/1-16', 'TapeEcho_Bridge-Middle/1-17', 'TapeEcho_Bridge-Middle/1-18', 'TapeEcho_Bridge-Middle/1-19', 'TapeEcho_Bridge-Middle/1-2', 'TapeEcho_Bridge-Middle/1-20', 'TapeEcho_Bridge-Middle/1-21', 'TapeEcho_Bridge-Middle/1-22', 'TapeEcho_Bridge-Middle/1-3', 'TapeEcho_Bridge-Middle/1-4', 'TapeEcho_Bridge-Middle/1-5', 'TapeEcho_Bridge-Middle/1-6', 'TapeEcho_Bridge-Middle/1-7', 'TapeEcho_Bridge-Middle/1-8', 'TapeEcho_Bridge-Middle/1-9', 'TapeEcho_Bridge-Middle/2-23', 'TapeEcho_Bridge-Middle/2-24', 'TapeEcho_Bridge-Middle/2-25', 'TapeEcho_Bridge-Middle/2-26', 'TapeEcho_Bridge-Middle/2-27', 'TapeEcho_Bridge-Middle/2-28', 'TapeEcho_Bridge-Middle/2-29', 'TapeEcho_Bridge-Middle/2-30', 'TapeEcho_Bridge-Middle/2-31', 'TapeEcho_Bridge-

In [8]:

effectType = ['Plate Reverb', 'RAT', 'Flanger', 'Phaser', 'Chorus', 'BluesDriver', 'Hall Reverb', 'TubeScreamer', 'Spring Reverb', 'Clean']

import librosa
import numpy as np
import mirdata
import soundfile as sf

guitarSounds = mirdata.initialize('egfxset')
guitarSounds_ids = guitarSounds.track_ids
# guitarSounds.download(['bluesDriver', 'chorus', 'clean', 'flanger', 'hallReverb', 'phaser', 'plateReverb', 'rat', 'spring-Reverb','tubeScreamer', 'metadata'], force_overwrite=True, cleanup=True, allow_invalid_checksum=True)


fade_duration = 0.05
fadein_samples = int(fade_duration/5 * 48000)
fadeout_samples = int(fade_duration/5 * 48000)

fade_in_window = np.linspace(0, 1, fadein_samples)
fade_out_window = np.linspace(1, 0, fadeout_samples)

import os
import jams
import random
from random import randint
random.seed(42)


folder = 'jams/'

guitarStrings = [40, 45, 50, 55, 59, 64]
stringNumber = [6, 5, 4, 3, 2, 1]  

def jamsToAudios(jams_file_path, output_dir):
    dur = 0
    # Load the JAMS file
    jam_data = jams.load(jams_file_path)
    duration = jam_data.file_metadata.duration
    num_samples = int(duration * 48000)
    string_audio = np.zeros((6, num_samples))
    
    for annotation in jam_data['annotations']:
        string = annotation['annotation_metadata']['data_source']
        namespace = annotation['namespace']
        if namespace == 'note_midi':
                        
            for value in annotation['data']:
                
                randomnumber = randint(0, 4)
                effectTypeSelection = randint(0,9)                

                bridgeTracks = [x for x in guitarSounds_ids if effectType[effectTypeSelection] + "_Bridge/" in x]
                bridgeMiddleTracks = [x for x in guitarSounds_ids if effectType[effectTypeSelection] + "_Bridge-Middle/" in x]
                middleTracks = [x for x in guitarSounds_ids if effectType[effectTypeSelection] + "_Middle/" in x]
                middleNeckTracks = [x for x in guitarSounds_ids if effectType[effectTypeSelection] + "_Middle-Neck/" in x]
                neckTracks = [x for x in guitarSounds_ids if effectType[effectTypeSelection] + "_Neck/" in x]

                pickupSelector = [bridgeTracks, bridgeMiddleTracks, middleTracks, middleNeckTracks, neckTracks]
                
                time = value[0]
                duration = value[1]
                note = value[2]
                fret = note - guitarStrings[int(string)]
                
                # print(string, time, fret, duration)
            
                # Find the corresponding string number
                string_num = int(stringNumber[int(string)]) - 1
                
                # Calculate start and end samples for the audio segment
                start_sample = int(time * 48000)
                end_sample = int((time + duration) * 48000)
                # print(start_sample, end_sample, num_samples)
                
                    
                if effectType[effectTypeSelection] == 'Flanger' and 'Bridge-Middle' in pickupSelector[randomnumber][0] and str(stringNumber[int(string)]) == '3':
                    # print('here')
                    track = [x for x in pickupSelector[randomnumber] if str(stringNumber[int(string)]) + '.' + str(round(fret)) in x]
                elif effectType[effectTypeSelection] == 'TubeScreamer' and 'Bridge-Middle' in pickupSelector[randomnumber][0] and str(stringNumber[int(string)]) == '1' and str(round(fret)) == '17':
                    # print('here')
                    track = [x for x in pickupSelector[2] if str(stringNumber[int(string)]) + '.' + str(round(fret)) in x]
                else:
                    track = [x for x in pickupSelector[randomnumber] if str(stringNumber[int(string)]) + '-' + str(round(fret)) in x]
                # print(effectType[effectTypeSelection] , randomnumber, str(stringNumber[int(string)]) + '-' + str(round(fret)), track)
                sound = guitarSounds.track(track[0]).audio_path
                # print(track)
                y, sr = librosa.load(sound, sr=48000)
                max_amplitude = np.max(np.abs(y))
                
                threshold = 0.5
                if effectType[effectTypeSelection] == 'RAT' or 'BluesDriver':
                    threshold = 0.25

                # Scale the audio to fit within the threshold
                scaling_factor = threshold / max_amplitude
                y = y * scaling_factor
            
                audio_cut = y[:end_sample - start_sample]
                # print("Audio Cut Shape:", audio_cut.shape)
                
                # Apply fade-in and fade-out
                audio_cut[:fadein_samples] *= fade_in_window
                audio_cut[-fadeout_samples:] *= fade_out_window
                
                
                # Insert the audio segment into the string_audio array
                randAmp = randint(60,90)/100
                audio_cut = audio_cut * randAmp
                
                if end_sample - start_sample > len(audio_cut):
                    length_factor = len(audio_cut)/(end_sample - start_sample)
                    audio_cut = librosa.effects.time_stretch(y,rate=length_factor)
                    
                slice_shape = string_audio[string_num, start_sample:end_sample].shape
                slice_length = end_sample - start_sample

                print("Slice shape:", slice_shape)

                # Compare the shape with the shape of audio_cut
                print("Audio cut shape:", audio_cut.shape)
                
                slice_shape = string_audio[string_num, start_sample:end_sample].shape

                # Calculate the length of the slice from its shape
                slice_length = slice_shape[0]

                # Check if audio_cut is one sample larger than the slice
                if len(audio_cut) == slice_length + 1:
                    # Truncate audio_cut by removing the last sample
                    audio_cut = audio_cut[:-1]

                # Now, assign audio_cut to the slice in string_audio
                string_audio[string_num, start_sample:start_sample + slice_length] = audio_cut
                                    
                            
                # string_audio[string_num, start_sample:end_sample] = audio_cut

                    
    polyphonic_audio = np.sum(string_audio, axis=0)
    
    normalized_audio = librosa.util.normalize(polyphonic_audio)
    file_name = os.path.splitext(os.path.basename(jams_file_path))[0]
    
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Save the polyphonic audio as a WAV file
    output_file_path = os.path.join(output_dir, file_name + '.wav')
    sf.write(output_file_path, normalized_audio, 48000, 'PCM_24')

    
    return output_file_path
               

# List files in the directory
file_list = os.listdir(folder)


for file_name in file_list:
    # Construct full file path
    file_path = os.path.join(folder, file_name)
    
    # Check if the path corresponds to a file (not a directory)
    if os.path.isfile(file_path):
        output_file_path = os.path.join('GuitarSetRandomBPM_JamsGeneratedAudio', file_name.split('.')[0]+'.wav')  # Assuming file_name is unique
        print(output_file_path)
        
        # Check if the output file already exists
        if os.path.exists(output_file_path):
            print("Skipping already created file:", file_name.split('.')[0]+'.wav')
            continue  # Skip to the next iteration
        
        print("Processing:", file_name)
        jamsToAudios(file_path, 'audios')

GuitarSetRandomBPM_JamsGeneratedAudio\1.wav
Processing: 1.jams


C:\Users\LENOVO LEGION  Y720\AppData\Local\Temp\ipykernel_2396\1175788816.py:88: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(sound, sr=48000)
C:\Users\LENOVO LEGION  Y720\AppData\Roaming\Python\Python311\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '/tmp\\mir_datasets\\egfxset\\Plate Reverb/Bridge/6-5.wav'